In [ ]:
import numpy as np
import pandas as pd
import os
import torchvision


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train = '/content/drive/MyDrive/ColabNotebooks/PokemonData'




Mounted at /content/drive


#ResNet50

In [ ]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import torch

resnet50 = models.resnet50(pretrained = True)

for param in resnet50.parameters():   #가중치 freeze
  param.requires_grad = False

num_classes = 150
num_ftrs = resnet50.fc.in_features

resnet50.fc = nn.Linear(num_ftrs, num_classes)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet50.to(device)

criterion = nn.CrossEntropyLoss() 

optimizer_ft = optim.SGD(resnet50.parameters(), lr=0.001,momentum=0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size =7, gamma=0.1)

In [ ]:
from tqdm import tqdm

def train(dataloader, model, loss_fn, optimizer):
    model.train()
    pbar = tqdm(enumerate(dataloader), total = len(dataloader)) #
    learning_loss = 0
    sum_loss = 0

    for batch, (X, y) in pbar:
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        sum_loss = sum_loss + loss.item() #
        learning_loss = sum_loss / (batch + 1)
        
        description = f'train loss : {round(learning_loss, 4)}'
        pbar.set_description(description)

def test(dataloader, model, loss_fn):
    model.eval()  #
    pbar = tqdm(enumerate(dataloader), total = len(dataloader)) #
    learning_loss = 0
    sum_loss = 0
    learning_acc = 0
    sum_acc = 0
    correct = 0

    for batch, (X, y) in pbar:
      X, y = X.to(device), y.to(device)

      pred = model(X)
      loss = loss_fn(pred, y)
      correct = (pred.argmax(1)==y).type(torch.float).sum().item()
      acc = correct / batch_size
      sum_acc += acc
      learning_acc = sum_acc / (batch + 1)
      sum_loss = sum_loss + loss.item() #
      learning_loss = sum_loss / (batch + 1)

      description = f'test loss : {learning_loss}  || test acc : {learning_acc}'
      pbar.set_description(description)

    return learning_acc

In [ ]:
epochs = 20
best_acc = 0

for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(trainloader,resnet50,criterion,optimizer_ft)
  acc = test(testloader,resnet50,criterion)
  if acc > best_acc:
    best_acc = acc
    torch.save(resnet50.state_dict(),f'/content/drive/MyDrive/ColabNotebooks/maskmodel_{acc}.pt') #(모델의 현재 파라미터저장,저장경로)